# Visualization of Test Suite Data
    This notebook visualizes the data generated by the test suite. It reads the output files and creates visualizations based on the data.

In [4]:
import os
import pandas as pd

def extract_info(output):
    """Extract information from synthesis output."""
    lines = output.replace(',', '\n').split('\n')
    info = {}
    capture = False
    for line in lines:
        if "Synthesis summary:" in line:
            capture = True
        if capture:
            if "constraint" in line:
                info["Constraint"] = line.split(": ")[1]
            elif "optimality objective" in line:
                info["Optimality Objective"] = line.split(": ")[1]
            elif "method" in line:
                info["Method"] = line.split(": ")[1]
            elif "synthesis time" in line:
                info["Synthesis Time"] = float(line.split(": ")[1].replace(' s', ''))
            elif "number of holes" in line:
                info["Number of Holes"] = int(line.split(": ")[1])
            elif "family size" in line:
                info["Family Size"] = line.split(": ")[1]  # Keep as string for scientific notation
            elif "quotient" in line:
                info["Quotient"] = line.split(": ")[1]
            elif "explored" in line:
                info["Explored Percentage"] = float(line.split(": ")[1].replace(' %', ''))
            elif "MDP stats" in line:
                info["MDP Stats"] = line.split(": ")[1]
            elif "optimum" in line:
                info["Optimum"] = float(line.split(": ")[1])
    return info

def create_latex_table(data, project, fsc_size):
    """Create a LaTeX table for given project and FSC size."""
    # Escape underscores in project name and fsc_size for LaTeX
    project_tex = project.replace('_', '\\_')
    fsc_size_tex = fsc_size.replace('_', '\\_')
    
    latex_table = (
        "\\begin{table}[h]\n"
        "\\centering\n"
        f"\\caption{{{project_tex} with {fsc_size_tex}}}\n"
        "\\begin{tabular}{|l|r|r|r|r|}\n"
        "\\hline\n"
        "Memory Constraint & Synthesis Time (s) & Family Size & Explored (\\%) & Optimum \\\\\n"
        "\\hline\n"
    )
    
    for _, row in data.iterrows():
        # Escape underscores in memory constraint names
        memory_constraint = row['Memory Constraint'].replace('_', '\\_')
        latex_table += (
            f"{memory_constraint} & "
            f"{row['Synthesis Time']:.2f} & "
            f"{row['Family Size']} & "
            f"{row['Explored Percentage']:.0f} & "
            f"{row['Optimum']:.6f} \\\\\n"
        )
    
    latex_table += (
        "\\hline\n"
        "\\end{tabular}\n"
        # Also escape underscores in the label
        "\\label{tab:" + f"{project_tex} {fsc_size_tex}" + "}\n"
        "\\end{table}\n"
    )
    return latex_table

# Set up directories
output_dir = "../test_outputs"
tables_dir = "../tables"
if not os.path.exists(tables_dir):
    os.makedirs(tables_dir)

# Initialize empty list to store data
rows = []

# Process test outputs
for project in os.listdir(output_dir):
    project_path = os.path.join(output_dir, project)
    if os.path.isdir(project_path):
        for fsc_size in os.listdir(project_path):
            fsc_path = os.path.join(project_path, fsc_size)
            if os.path.isdir(fsc_path):
                for memory_constraint in os.listdir(fsc_path):
                    memory_path = os.path.join(fsc_path, memory_constraint)
                    output_file = os.path.join(memory_path, "output.txt")
                    if os.path.isfile(output_file):
                        with open(output_file, 'r') as f:
                            output = f.read()
                        info = extract_info(output)
                        info['Project'] = project
                        info['FSC Size'] = fsc_size
                        info['Memory Constraint'] = memory_constraint
                        rows.append(info)

# Create DataFrame
df = pd.DataFrame(rows)

# Generate LaTeX tables
for project in df['Project'].unique():
    project_dir = os.path.join(tables_dir, project)
    if not os.path.exists(project_dir):
        os.makedirs(project_dir)
    
    for fsc_size in df[df['Project'] == project]['FSC Size'].unique():
        mask = (df['Project'] == project) & (df['FSC Size'] == fsc_size)
        subset = df[mask].copy()
        
        table_content = create_latex_table(subset, project, fsc_size)
        
        # Save the table
        filename = f"{fsc_size}.tex"
        filepath = os.path.join(project_dir, filename)
        with open(filepath, 'w') as f:
            f.write(table_content)
        
        print(f"Created table for {project} - {fsc_size}")

print("\nAll tables have been generated.")

Created table for maze-mo - fsc_size_5
Created table for maze-mo - fsc_size_4
Created table for maze-mo - fsc_size_2
Created table for maze-mo - fsc_size_3
Created table for maze-mo - fsc_size_7
Created table for maze-mo - fsc_size_6
Created table for grid-avoid-4-0.1 - fsc_size_5
Created table for grid-avoid-4-0.1 - fsc_size_4
Created table for grid-avoid-4-0.1 - fsc_size_2
Created table for grid-avoid-4-0.1 - fsc_size_3
Created table for grid-avoid-4-0.1 - fsc_size_7
Created table for grid-avoid-4-0.1 - fsc_size_6
Created table for grid-large-12-4 - fsc_size_5
Created table for grid-large-12-4 - fsc_size_4
Created table for grid-large-12-4 - fsc_size_2
Created table for grid-large-12-4 - fsc_size_3
Created table for grid-large-12-4 - fsc_size_7
Created table for grid-large-12-4 - fsc_size_6
Created table for grid-avoid-4-0.1-goal-in-center - fsc_size_5
Created table for grid-avoid-4-0.1-goal-in-center - fsc_size_4
Created table for grid-avoid-4-0.1-goal-in-center - fsc_size_2
Created

In [5]:
def create_latex_documents():
    # Base LaTeX header
    latex_header = [
        "\\documentclass{article}",
        "\\usepackage[margin=1in]{geometry}",
        "\\usepackage{booktabs}",
        "\\usepackage{multirow}",
        "\\begin{document}",
        "\\title{Synthesis Results}"
    ]
    
    # Walk through the tables directory
    tables_dir = "../tables"
    for project in sorted(os.listdir(tables_dir)):
        project_path = os.path.join(tables_dir, project)
        if os.path.isdir(project_path):
            # Create content for this project
            latex_content = latex_header.copy()
            latex_content.extend([
                f"\\author{{{project}}}",
                "\\maketitle",
                f"\\section{{{project}}}"
            ])
            
            # Add all tables for this project
            for table_file in sorted(os.listdir(project_path)):
                if table_file.endswith('.tex'):
                    with open(os.path.join(project_path, table_file), 'r') as f:
                        latex_content.append(f.read())
                    latex_content.append("") # Add empty line for spacing
            
            # Close the document
            latex_content.append("\\end{document}")
            
            # Write the LaTeX document for this project
            tex_file = os.path.join(project_path, f"{project}_results.tex")
            with open(tex_file, 'w') as f:
                f.write("\n".join(latex_content))
            
            

# Generate the documents
create_latex_documents()